In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
import os
os.makedirs('/root/.kaggle',exist_ok = True)
!cp /content/gdrive/My\ Drive/DeepLearning/kaggle/kaggle.json /root/.kaggle/kaggle.json

In [0]:
!kaggle competitions download -c the-nature-conservancy-fisheries-monitoring

In [0]:
!kaggle datasets download -d ningyue1/alexnet1

In [0]:
# 解压预训练权重
! unzip /content/alexnet1.zip

In [0]:
# 解压训练集数据
! unzip /content/train.zip

In [0]:
# 解压第一部分测试集数据
!unzip /content/test_stg1.zip

In [0]:
# 解压第二部分测试集数据
!7za x test_stg2.7z

In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import os,cv2
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [0]:
# 定义取图片的路径
TRAIN_DIR = '/content/train/'
TEST_DIR = '/content/test_stg2/'
WEIGHT_DIR = '/content/bvlc_alexnet.npy'
FISH_CLASSES = ['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']

In [0]:
# 获取图片的路径方法
def get_images(data_dir,fish):
    """Load files from train folder"""
    fish_dir = data_dir+'{}'.format(fish)
    images = [fish+'/'+im for im in os.listdir(fish_dir)]
    return images
# 将图片读入内存
def read_image(src):
    """Read and resize individual images"""
    im = cv2.imread(src, cv2.IMREAD_COLOR)
    im = cv2.resize(im, (227, 227), interpolation=cv2.INTER_CUBIC)
    return im

In [0]:
# 获取训练数据
files = []
y_all = []

for fish in FISH_CLASSES:
    fish_files = get_images(TRAIN_DIR,fish)
    files.extend(fish_files)
    
    y_fish = np.tile(fish, len(fish_files))
    y_all.extend(y_fish)
    print("{0} photos of {1}".format(len(fish_files), fish))
    
y_all = np.array(y_all)

1719 photos of ALB
200 photos of BET
117 photos of DOL
67 photos of LAG
465 photos of NoF
299 photos of OTHER
176 photos of SHARK
734 photos of YFT


In [0]:
# 将所有图片加载到内存中
X = np.ndarray((len(files), 227, 227, 3))

for i, im in enumerate(files): 
    X[i] = read_image(TRAIN_DIR+im)
    if i%1000 == 0: print('Processed {} of {}'.format(i, len(files)))
print(X.shape)

Processed 0 of 3777
Processed 1000 of 3777
Processed 2000 of 3777
Processed 3000 of 3777
(3777, 227, 227, 3)


In [0]:
# 转化为one-hot编码
y = LabelEncoder().fit_transform(y_all)
y = np_utils.to_categorical(y)

In [0]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                    test_size=0.2, random_state=23, 
                                                    stratify=y)

In [0]:
print(X_train.shape,X_valid.shape,y_train.shape,y_valid.shape)

(3021, 227, 227, 3) (756, 227, 227, 3) (3021, 8) (756, 8)


In [0]:
# 加载预训练的权重
pre_weight = np.load(WEIGHT_DIR,encoding='bytes',allow_pickle=True).item()

In [0]:
# 学习参数
learning_rate = 0.003
num_epochs = 50
batch_size = 64
batch_num = 47

# 网络参数
dropout_rate = 0.5
num_classes = 8
train_layers = ['fc8', 'fc7', 'fc6']

# 多久将tf.summary写入磁盘
display_step = 20

In [0]:
# 构建模型
x = tf.placeholder(tf.float32, [None, 227, 227, 3])
y = tf.placeholder(tf.int32, [None, num_classes])

W_conv = {
    'conv1': tf.Variable(pre_weight['conv1'][0],trainable=False),
    'conv2': tf.Variable(pre_weight['conv2'][0],trainable=False),
    'conv3': tf.Variable(pre_weight['conv3'][0],trainable=False),
    'conv4': tf.Variable(pre_weight['conv4'][0],trainable=False),
    'conv5': tf.Variable(pre_weight['conv5'][0],trainable=False),
    'fc1': tf.Variable(pre_weight['fc6'][0],trainable=False),
    'fc2': tf.Variable(tf.truncated_normal([4096, 4096], stddev=0.1)),
    'fc3': tf.Variable(tf.truncated_normal([4096, num_classes], stddev=0.1))
}
b_conv = {
    'conv1': tf.Variable(pre_weight['conv1'][1],trainable=False),
    'conv2': tf.Variable(pre_weight['conv2'][1],trainable=False),
    'conv3': tf.Variable(pre_weight['conv3'][1],trainable=False),
    'conv4': tf.Variable(pre_weight['conv4'][1],trainable=False),
    'conv5': tf.Variable(pre_weight['conv5'][1],trainable=False),
    'fc1': tf.Variable(pre_weight['fc6'][1],trainable=False),
    'fc2': tf.Variable(tf.constant(0.1, dtype=tf.float32, shape=[4096])),
    'fc3': tf.Variable(tf.constant(0.0, dtype=tf.float32, shape=[num_classes]))
}

In [0]:
# 卷积层 1
conv1 = tf.nn.conv2d(x, W_conv['conv1'], strides=[1, 4, 4, 1], padding='VALID')
conv1 = tf.nn.bias_add(conv1, b_conv['conv1'])
conv1 = tf.nn.relu(conv1)
# LRN层，Local Response Normalization
norm1 = tf.nn.lrn(conv1, 5, bias=1.0, alpha=0.001 / 9.0, beta=0.75)
# 池化层 1
pool1 = tf.nn.max_pool(norm1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='VALID')

#卷积层 2
conv2 = tf.nn.conv2d(pool1, W_conv['conv2'], strides=[1, 1, 1, 1], padding='SAME')
conv2 = tf.nn.bias_add(conv2, b_conv['conv2'])
conv2 = tf.nn.relu(conv2)
# LRN层，Local Response Normalization
norm2 = tf.nn.lrn(conv2, 5, bias=1.0, alpha=0.001 / 9.0, beta=0.75)
# 池化层 2
pool2 = tf.nn.max_pool(norm2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='VALID')

# 卷积层3
conv3 = tf.nn.conv2d(pool2, W_conv['conv3'], strides=[1, 1, 1, 1], padding='SAME')
conv3 = tf.nn.bias_add(conv3, b_conv['conv3'])
conv3 = tf.nn.relu(conv3)

# 卷积层4
conv4 = tf.nn.conv2d(conv3, W_conv['conv4'], strides=[1, 1, 1, 1], padding='SAME')
conv4 = tf.nn.bias_add(conv4, b_conv['conv4'])
conv4 = tf.nn.relu(conv4)

# 卷积层5
conv5 = tf.nn.conv2d(conv4, W_conv['conv5'], strides=[1, 1, 1, 1], padding='SAME')
conv5 = tf.nn.bias_add(conv5, b_conv['conv5'])
conv5 = tf.nn.relu(conv5)
# 池化层5
pool5 = tf.nn.max_pool(conv5, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='VALID')

reshape = tf.reshape(pool5, [-1, 6 * 6 * 256])

fc1 = tf.add(tf.matmul(reshape, W_conv['fc1']), b_conv['fc1'])
fc1 = tf.nn.relu(fc1)
fc1 = tf.nn.dropout(fc1, 0.5)
# 全连接层 2
fc2 = tf.add(tf.matmul(fc1, W_conv['fc2']), b_conv['fc2'])
fc2 = tf.nn.relu(fc2)
fc2 = tf.nn.dropout(fc2, 0.5)
# 全连接层 3, 即分类层
fc3 = tf.add(tf.matmul(fc2, W_conv['fc3']), b_conv['fc3'])

W0716 07:21:23.151673 140263705421696 deprecation.py:506] From <ipython-input-13-58eeb1ebcff1>:39: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
# 定义损失
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=fc3, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss)
# 评估模型
correct_pred = tf.equal(tf.argmax(fc3, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [0]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for i in range(num_epochs):
        for j in range(batch_num):
            X_train_data = X_train[j*batch_size:(j+1)*batch_size,:,:,:]
            y_train_data = y_train[j*batch_size:(j+1)*batch_size,:]
            sess.run(optimizer,feed_dict={x:X_train_data,y:y_train_data})
            if j % 10 ==0:  
                _loss = sess.run(loss, feed_dict={x:X_train_data,y:y_train_data})
                train_accuracy = sess.run(accuracy, feed_dict={x:X_train_data, y: y_train_data})
                print('Epoch: {:>2} Batch {:>5} Loss: {:>10.6f} Accuracy: {:.6f}'.format(i, j, _loss, train_accuracy))

Epoch:  0 Batch     0 Loss:  44.370239 Accuracy: 0.484375
Epoch:  0 Batch    10 Loss:  49.111614 Accuracy: 0.390625
Epoch:  0 Batch    20 Loss:  28.110809 Accuracy: 0.593750
Epoch:  0 Batch    30 Loss:  33.147087 Accuracy: 0.500000
Epoch:  0 Batch    40 Loss:  22.182247 Accuracy: 0.531250
Epoch:  1 Batch     0 Loss:  26.174402 Accuracy: 0.531250
Epoch:  1 Batch    10 Loss:  31.976316 Accuracy: 0.531250
Epoch:  1 Batch    20 Loss:  28.280945 Accuracy: 0.609375
Epoch:  1 Batch    30 Loss:  18.346161 Accuracy: 0.531250
Epoch:  1 Batch    40 Loss:  27.141014 Accuracy: 0.531250
Epoch:  2 Batch     0 Loss:  17.594864 Accuracy: 0.593750
Epoch:  2 Batch    10 Loss:  18.017454 Accuracy: 0.484375
Epoch:  2 Batch    20 Loss:  17.151209 Accuracy: 0.640625
Epoch:  2 Batch    30 Loss:  22.497936 Accuracy: 0.515625
Epoch:  2 Batch    40 Loss:  19.007866 Accuracy: 0.578125
Epoch:  3 Batch     0 Loss:  16.737545 Accuracy: 0.625000
Epoch:  3 Batch    10 Loss:  21.045330 Accuracy: 0.531250
Epoch:  3 Batc

In [0]:
import gc
del X
gc.collect()

In [0]:
 test_file_path = [TEST_DIR+im for im in os.listdir(TEST_DIR)]

In [0]:
test_preds = np.zeros((12153,8))

In [0]:
test = np.ndarray((100, 227, 227, 3))

In [0]:
with tf.Session() as sess:
    sess.run(init)
    for i in range(121):
      print(i)
      for j in range(100):
        test[j] = read_image(test_file_path[i*100+j])
      t_pred = sess.run(tf.nn.softmax(fc3),feed_dict={x:test})
      test_preds[i*100:(i+1)*100,:] = t_pred
    for i in range(53):
      test[j] = read_image(test_file_path[121*100+i])
    t_pred = sess.run(tf.nn.softmax(fc3),feed_dict={x:test[0:53]})
    test_preds[121*100:121*100+53,:] = t_pred

In [0]:
test_file_path1 = ['test_stg2/'+im for im in os.listdir(TEST_DIR)]

In [0]:
submission = pd.DataFrame(test_preds, columns=FISH_CLASSES)
submission.insert(0, 'image', test_file_path1)
submission.head()


submission.to_csv('submisision2.csv',index=None)


In [0]:
submission.head()

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,test_stg2/image_09747.jpg,9.440124e-36,1.000000e+00,2.653607e-26,5.955861e-27,6.568304e-30,2.043091e-19,3.372094e-38,6.370285e-12
1,test_stg2/image_04623.jpg,0.000000e+00,2.144653e-22,0.000000e+00,1.912081e-32,1.000000e+00,0.000000e+00,1.615676e-19,7.727690e-10
2,test_stg2/image_09181.jpg,2.972477e-05,1.920969e-36,2.324049e-10,0.000000e+00,3.381241e-31,0.000000e+00,1.474657e-36,9.999703e-01
3,test_stg2/image_07560.jpg,0.000000e+00,1.269314e-13,1.000000e+00,0.000000e+00,1.459537e-25,0.000000e+00,1.104047e-25,0.000000e+00
4,test_stg2/image_12085.jpg,0.000000e+00,1.359807e-37,1.203263e-12,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [0]:
test_files = [im for im in os.listdir('/content/test_stg1')]
test = np.ndarray((len(test_files), 227, 227, 3))
for i, im in enumerate(test_files): 
    test[i] = read_image('/content/test_stg1/'+im)

In [0]:
test_preds = np.zeros((1000,8))
with tf.Session() as sess:
    sess.run(init)
    for i in range(10):
        print(i)
        t_pred = sess.run(tf.nn.softmax(fc3),feed_dict={x:test[i*100:(i+1)*100,:,:,:]})
        test_preds[i*100:(i+1)*100,:] = t_pred



submission = pd.DataFrame(test_preds, columns=FISH_CLASSES)
submission.insert(0, 'image', test_files)
submission.head()


submission.to_csv('submisision1.csv',index=None)


0
1
2
3
4
5
6
7
8
9
